# Lesson 4: Creating Relationships Between Models

## Creating Relationships Between Models

## Add a Priority Model

## Fix the Relationships in Models

## Creating Song Model

## Creating Relationships Between Models